## Agents

### 🤖 What is an Agent?

- An LLM configured with specific instructions to do a task. It may use tools to achieve more specific goals.

#### install

In [ ]:
# ! pip install openai-agents nest_asyncio


### imports

In [25]:
from pydantic import BaseModel
import dotenv
import nest_asyncio
from agents import Agent, Runner, GuardrailFunctionOutput, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import asyncio
from typing import AsyncGenerator
import requests
import json
from IPython.display import display, Markdown, DisplayHandle
import os

##### Apply nest_asyncio to allow asyncio event loops to be nested

In [3]:
nest_asyncio.apply()

#### Load environment variables

In [26]:
dotenv.load_dotenv()

GPT_4_1_NANO = os.getenv('CHEAP_AND_BEST_MODEL', 'gpt-4.1-nano')

##### function tool

In [65]:
@function_tool
def get_usd_base_rate() -> dict:
  """
  Get today's USD base rate.
  """
  response: dict = requests.get("https://api.vatcomply.com/rates?base=USD")
  if response.status_code != 200:
    return f"Error: {response.status_code}"
  return response.json()

### Agent with tool

In [66]:
currency_agent = Agent(
  name="Currency Agent",
  model=GPT_4_1_NANO,
  tools=[get_usd_base_rate],
  instructions="""You are an expert financial assistant, with expertise in currency conversion. To get the latest USD base rate, use the get_usd_base_rate function.
Provide the user with answers in their requested currency and USD equivalent.

Example: 
User: How much does it cost to buy a tesla in London?
Assistant: The cost of a Tesla in London is £50,000. This is equivalent to $65,000 USD based on the current exchange rate.
"""
)

### Ask the agent

In [67]:
async def ask_agent(q: str, agent: Agent, context: object = None) -> AsyncGenerator[str, None]:      
    result = Runner.run_streamed(agent, input=q, context=context)
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            yield event.data.delta

##### question

In [68]:
question: str = "How much does it cost to buy a tesla in Canada?"

#### Streaming Response (with Markdown)

In [69]:
async def stream_response(q: str, agent: Agent, context: Any = None) -> AsyncGenerator[str, None]:
    """
    Stream the response from the agent.
    """
    full_response = ""
    display_handle = display(Markdown(full_response), display_id=True)
    async for chunk in ask_agent(q, agent, context):
      full_response += chunk
      display_handle.update(Markdown(full_response))
      await asyncio.sleep(0.01)

In [70]:
asyncio.run(stream_response(question, currency_agent))

The current exchange rate is approximately 1 USD = 1.39 CAD. The price of a Tesla in Canada varies, but let's assume it costs around CAD 60,000. This would be roughly equivalent to about $43,165 USD based on the current exchange rate.

### Handoffs

##### What is a handoff 🔄
- A mechanism that allows agents to transfer tasks to one or more other agents.
- LLMs perceive handoffs just like any other tool.

##### NOTE: 
- A common pattern includes an initial triage/routing agent followed by specialized agents, with optional escalation to a human or more advanced agent.

##### Example: Handoffs in Travel Agents

A handoff allows an agent to transfer a user's request to another specialized agent. In the travel domain, this could mean routing a query to a flight booking agent, hotel booking agent, or itinerary planner.

In [71]:
from agents import Agent, handoff

flight_agent = Agent(
  name="Flight Booking Agent", 
  instructions="Book flights and answer flight-related queries.",
  model=GPT_4_1_NANO
  )
hotel_agent = Agent(
  name="Hotel Booking Agent", instructions="Book hotels and answer hotel-related queries.",
  model=GPT_4_1_NANO
)
itinerary_agent = Agent(
  name="Itinerary Planner Agent", instructions="Plan travel itineraries and suggest activities.",
  model=GPT_4_1_NANO
)

travel_triage_agent = Agent(
    name="Travel Triage Agent",
    instructions="Route the user's request to the appropriate travel agent: flight, hotel, or itinerary.",
    handoffs=[
        handoff(flight_agent),
        handoff(hotel_agent),
        handoff(itinerary_agent)
    ],
    model=GPT_4_1_NANO
)

### Example: Routing a User Query
Let's see how a user's request is routed to the right agent.

In [73]:
async def ask_travel_ai(q: str):
    result = await Runner.run(travel_triage_agent, q)
    print(result.final_output)

# Example queries
asyncio.run(ask_travel_ai("Book a flight from New York to London"))
asyncio.run(ask_travel_ai("Find a hotel in Paris"))
asyncio.run(ask_travel_ai("Plan a 3-day itinerary for Tokyo"))

I'll assist you with booking a flight from New York to London. Could you please provide me with the preferred dates of travel, class of service (economy, business, first class), and any other specific preferences?
I can help you find a hotel in Paris. Could you please tell me your preferred check-in and check-out dates, the number of guests, and any specific preferences or budget?
I can help you find a hotel in Paris. Could you please tell me your preferred check-in and check-out dates, the number of guests, and any specific preferences or budget?
Certainly! Here's a 3-day itinerary for Tokyo that balances popular attractions, cultural experiences, and local flavor:

---

### **Day 1: Classic Tokyo & Cultural Highlights**
**Morning:**
- **Asakusa & Senso-ji Temple**: Start your day exploring Tokyo's oldest temple and the vibrant Nakamise shopping street for souvenirs and traditional snacks.
- **Sumida River Cruise**: Enjoy a scenic boat ride from Asakusa to Odaiba, offering great views

### Example: Streaming Response
Let's see how a triage agent can stream responses while routing travel-related queries to the appropriate specialized agent.

In [74]:
asyncio.run(stream_response(agent=travel_triage_agent, q="Book me a flight from New York to Tokyo next week."))
asyncio.run(stream_response(agent=travel_triage_agent, q="Find a hotel in Paris for 3 nights."))
asyncio.run(stream_response(agent=travel_triage_agent, q="Plan a 4-day itinerary in Rome."))

I will now assist you with booking a flight from New York to Tokyo for next week. Please specify your preferred travel dates and any other preferences such as your class of service (economy, business, etc.).

I can help you find a hotel in Paris for 3 nights. Do you have any preferences regarding the hotel type, location, or budget?

Certainly! Here's a suggested 4-day itinerary for Rome, balancing historical sites, cultural experiences, and leisure:

---

### Day 1: Ancient Rome & Colosseum
- **Morning:**  
  - Visit the **Colosseum** (buy tickets in advance)  
  - Explore the nearby **Roman Forum** and **Palatine Hill**  

- **Afternoon:**  
  - Walk through **Via dei Fori Imperiali**  
  - Visit **Piazza Venezia** and **Altare della Patria**  

- **Evening:**  
  - Relax at **Piazza Navona**  
  - Dinner in the area, try Roman specialties like carbonara or saltimbocca  

---

### Day 2: Vatican City
- **Morning:**  
  - Tour the **Vatican Museums** (including the Sistine Chapel) — pre-book tickets to avoid long lines  
  - Visit **St. Peter's Basilica** and climb to the dome for panoramic views  

- **Afternoon:**  
  - Walk along the **Apostolic Palace** gardens (if available)  
  - Explore **Castel Sant'Angelo**  

- **Evening:**  
  - Stroll along the Tiber River  
  - Enjoy dinner near Piazza del Popolo or Trastevere  

---

### Day 3: Artistic & Historic Rome
- **Morning:**  
  - Visit **Pantheon** early to avoid crowds  
  - Walk to **Piazza della Rotonda** and surrounding streets  

- **Afternoon:**  
  - Explore **Campo de' Fiori** and **Piazza Farnese**  
  - Visit **Borghese Gallery and Gardens** (advance reservation required)  

- **Evening:**  
  - Spend time in **Trastevere** neighborhood, known for its vibrant nightlife and traditional Roman food  

---

### Day 4: Hidden Gems & Leisure
- **Morning:**  
  - Explore **Aventine Hill**, including **Santa Sabina** and the **Keyhole of Rome** for a unique view of St. Peter’s Dome  
  - Walk through **Testaccio** neighborhood for authentic Roman cuisine  

- **Afternoon:**  
  - Relax in **Villa Borghese Park**  
  - Optional: Rent a bike or boat in the park  

- **Evening:**  
  - Enjoy a farewell dinner with a view at a rooftop restaurant, such as **Terrazza Borromini** or similar  

---

Would you like personalized restaurant recommendations, local tips, or assistance with booking tickets?

#### Dynamic Instructions

- To provide runtime instructions to the agent via the `dynamic_instructions` parameter. It can be of type regular function or an async function.
- The function should return a string that will be used as the instructions for the agent.

##### Example:

In [ ]:
from agents import RunContextWrapper, Agent
from pydantic import BaseModel

class UserContext(BaseModel):
    """
    User context for the travel agent.
    """
    name: str
    location: str
    preferences: dict

def dynamic_instructions(context: RunContextWrapper[UserContext], agent: Agent[UserContext]) -> str:
    """
    Generate dynamic instructions based on the context.
    """
    if context and context.context:
        user = context.context
        return f"""Greet the user with their name {user.name}. Assist the User from {user.location}, with their travel-related queries.
Consider their preferences: {', '.join([f'{k}: {v}' for k, v in user.preferences.items()])}."""
    else:
        return "Assist the user with their travel-related queries. Context was not provided."


triage_agent_with_dynamic_instructions: Agent = Agent[UserContext](
    name="Travel Triage Agent",
    model=GPT_4_1_NANO,
    instructions=dynamic_instructions
)

In [76]:
# Create a sample user context
user = UserContext(
    name="Alice",
    location="New York",
    preferences={
        "budget": "mid-range",
        "interests": ["culture", "food"],
        "travel_style": "solo"
    }
)
context = RunContextWrapper(user)
asyncio.run(stream_response(q='Travel to Madurai, Apr 2025', agent=triage_agent_with_dynamic_instructions, context=context.context))

Hello Alice! Planning a trip to Madurai in April 2025 sounds exciting. Madurai is rich in culture and famous for its stunning temples and vibrant food scene. 

Since you're traveling solo and prefer a mid-range budget, I can help you with:

- Best accommodations options
- Must-visit cultural sites
- Local cuisine and food experiences
- Travel tips for April in Madurai

Would you like me to suggest an itinerary, recommend accommodations, or provide details on transportation options?